In [1]:
# load graph and perform find-replace on defined terms.
# unused terms ("unknown") should be removed before this stage, and untranslated terms throw an error.
# for now classes are inferred by property, better would be explicit declarations in the XML -> TTL transformation.

import pathlib
import rdflib

def replace_entity(g, prop, rep):

    ''' Tedious find and replace process. '''

    countries = [o for s,p,o in g.triples((None, prop, None))]
    for x in countries:
        if x not in rep.keys():
            raise Exception(f'Equivalent term not found for {x}.')

    new_graph = rdflib.Graph()
    for s,p,o in g.triples((None, None, None)):
        if s in rep.keys():
            s = rep[s]
        if p in rep.keys():
            p = rep[p]
        if o in rep.keys():
            o = rep[o]
        new_graph.add((s,p,o))

    return new_graph

graph = rdflib.Graph()
graph.parse(pathlib.Path.cwd() / 'cd6685c5.ttl')

fiaf = rdflib.Namespace('https://fiafcore.org/ontology/')
graph.bind("fiaf", fiaf) 

# replace countries

country_replace = {
    rdflib.term.URIRef('https://www.bundesarchiv.de/country/%0A%09%09Bundesrepublik%20Deutschland%0A%09'): fiaf.Germany}

graph = replace_entity(graph, fiaf.hasCountry, country_replace)

# replace bases

base_replace = {
    rdflib.term.URIRef('https://www.bundesarchiv.de/base/%0A%09%09%09%09Triazetatzellulose%0A%09%09%09'): fiaf.Acetate,
    rdflib.term.URIRef('https://www.bundesarchiv.de/base/%0A%09%09%09%09Polyethylenterephtalat%20%28Polyester%29%0A%09%09%09'): fiaf.Polyester,
    
    # interesting, you would want to remove "unknown" before you get to this stage.
    rdflib.term.URIRef('https://www.bundesarchiv.de/base/%0A%09%09%09%09keiner%0A%09%09%09'): fiaf.Unknown,   
    
    }

graph = replace_entity(graph, fiaf.hasBase, base_replace)

# render harmonised graph.

graph.serialize(destination=pathlib.Path.cwd() / 'cd6685c5-harmonised.ttl', format='ttl')
print(len(graph), 'triples.')

306 triples.
